# Assignment Week 3 : Creating A DataFrame Containing Coordinates
## Author: Rahul Kulkarni

### Importing Necessary Libraries
We will make use of pandas to read the table from the wikipedia page

In [131]:
import pandas as pd
WIKI_URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
Wiki_Tables = pd.read_html(WIKI_URL, header=0)
#The link returns all the tables on the webpage in form of a list. Hence will slice the list and keep the table which is useful to us.
df = Wiki_Tables[0]
df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


### Dropping Unnecessary Rows
Since we require the name of the Borough, we will drop all the rows where the name is missing.

In [132]:
df.drop(df[df['Borough']=='Not assigned'].index,inplace=True)

We will reset the indexes and let's take a look at the dataframe we have created till now.

In [133]:
df.index = list(range(103))
df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Adding Latitudes and Longitudes
We will make use of GeoPy library to fetch the coordinates. But there are certain boroughs for which the function doesn't return anything i.e returns a None object. For these cases we will make use of the file containing the coordinates for the Postal Codes.

In [134]:
geo = pd.read_csv('D:/Geospatial_Coordinates.csv')

We will make use of the Borough, Neighbourhood and Postal Code to generate the coordinates.

In [135]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="foursquare_agent")
df['Latitude'],df['Longitude'] = 0,0
lat,long = [],[]
for b,n,i,p in zip(df['Borough'],df['Neighbourhood'],df.index,df['Postal Code']):
    address = n+','+b+',Toronto,Ontario'
    location = geolocator.geocode(address)
    if location is not None:
        lat.append(location.latitude)
        long.append(location.longitude)
        continue
    else:
        address = p+',Toronto,Ontario'
        location = geolocator.geocode(address)
        if location is not None:
            lat.append(location.latitude)
            long.append(location.longitude)
            continue
    if df.loc[i,'Latitude']==0:
        lati = float(geo[geo['Postal Code']==p]['Latitude'])
        longi = float(geo[geo['Postal Code']==p]['Longitude'])
        lat.append(lati)
        long.append(longi)

Let's assign the values we have collected to the dataframe.

In [136]:
df['Latitude'],df['Longitude'] = lat,long

In [137]:
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.761124,-79.324059
1,M4A,North York,Victoria Village,43.732658,-79.311189
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.716516,-79.442261
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.653482,-79.383935


In [138]:
df.to_csv('D:/Clean_Toronto.csv')